<a href="https://colab.research.google.com/github/Shreehari-Acharya/Quantum-computing-101/blob/main/shor's_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install qiskit qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 90.7 MB/s eta 0:00:00


In [9]:
from qiskit import QuantumCircuit, transpile
import numpy as np

def c_amod15(a, power):
    """Controlled multiplication by a mod 15."""
    if a not in [2, 7, 8, 11, 13]:
        raise ValueError("'a' must be 2, 7, 8, 11, or 13")
    U = QuantumCircuit(4)
    for _ in range(power):
        if a in [2, 13]:
            U.swap(0,1); U.swap(1,2); U.swap(2,3)
        if a in [7, 8]:
            U.swap(2,3); U.swap(1,2); U.swap(0,1)
        if a == 11:
            U.swap(1,3); U.swap(0,2)
        if a in [7, 11, 13]:
            for j in range(4): U.x(j)

    # Convert to a gate and return the controlled version
    U = U.to_gate()
    U.name = f"{a}^{power} mod 15"
    c_U = U.control()
    return c_U

In [4]:
from qiskit.circuit.library import QFTGate

def shor_circuit(n_count, a):
    # n_count: number of counting qubits
    # 4 qubits for the state register (since 15 < 2^4)
    qc = QuantumCircuit(n_count + 4, n_count)

    # Initialize counting qubits in superposition
    qc.h(range(n_count))

    # Auxiliary register must be in state |1>
    qc.x(n_count + 3)

    # Apply controlled modular exponentiation
    for q in range(n_count):
        qc.append(c_amod15(a, 2**q), [q] + [i + n_count for i in range(4)])

    # Apply inverse QFT
    qc.append(QFTGate(n_count).inverse(), range(n_count))

    # Measure the counting register
    qc.measure(range(n_count), range(n_count))
    return qc

# Example: Factor 15 using a=7
n_count = 8
qc = shor_circuit(n_count, a=7)

In [10]:
from qiskit_aer import AerSimulator
from math import gcd
from fractions import Fraction

# 1. Run the simulation
sampler = AerSimulator()
jobs = transpile(qc, sampler)

result = sampler.run(jobs, shots=1024).result()
counts = result.get_counts()

# 2. Process the output
measured_str = list(counts.keys())[0]
measured_int = int(measured_str, 2)
phase = measured_int / (2**n_count)

# 3. Find the period 'r' using continued fractions
frac = Fraction(phase).limit_denominator(15)
r = frac.denominator

# 4. Calculate factors
if r % 2 == 0:
    guesses = [gcd(7**(r//2) - 1, 15), gcd(7**(r//2) + 1, 15)]
    factors = [f for f in guesses if f not in [1, 15]]
    print(f"Measured Phase: {phase}")
    print(f"Predicted Period: {r}")
    print(f"Found Factors: {factors}")
else:
    print("Period found was odd, try a different 'a' value.")

Measured Phase: 0.5
Predicted Period: 2
Found Factors: [3]
